In [1]:
import os
import gzip
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime

In [2]:
# ==============================
# CONFIGURATION
# ==============================
OBS_CSV = "concentrations_with_err_fixed.csv"       # observation CSV
SRM_FILE = "SRM.nc"                                 # NetCDF SRM data
OUTDIR = "converted_input"                          # output folder
SCALING_FACTOR = 1e15                               # default scaling factor
os.makedirs(OUTDIR, exist_ok=True)

In [3]:
# ==============================
# LOAD DATA
# ==============================
obs = pd.read_csv(OBS_CSV, parse_dates=["COLLECT_START", "COLLECT_STOP"])
srm = xr.open_dataset(SRM_FILE)

lats_map = srm["lats_map"].values
lons_map = srm["lons_map"].values

ts_times = pd.to_datetime(srm["ts"].values)
td_times = pd.to_datetime(srm["td"].values)

n_sensors = srm.dims["sensor_i"]

/tmp/ipykernel_706527/77221451.py:13: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_sensors = srm.dims["sensor_i"]


In [4]:
# ==============================
# PROVIDED STATION COORDINATES
# ==============================
lons = [158.780000, -177.370000, 166.610861, -160.491367, -157.994972, -121.362500, -123.445388]
lats = [53.050000, 28.220000, 19.292278, 55.337133, 21.522444, 38.673333, 48.651305]

In [5]:
# ==============================
# ASSIGN DEFAULT STATION NAMES
# ==============================
station_names = [f"STA_{lat:.3f}_{lon:.3f}" for lat, lon in zip(lats, lons)]

station_names

['STA_53.050_158.780',
 'STA_28.220_-177.370',
 'STA_19.292_166.611',
 'STA_55.337_-160.491',
 'STA_21.522_-157.995',
 'STA_38.673_-121.362',
 'STA_48.651_-123.445']

In [6]:
# ==============================
# ASSIGN STATION NAMES TO OBSERVATIONS
# ==============================

def infer_station_names(obs_df, station_coords, station_names):
    """
    Assign station names to observations based on nearest coordinates.

    Parameters:
        obs_df (pd.DataFrame): Observation DataFrame with LAT, LON columns
        station_coords (list of tuples): [(lat1, lon1), (lat2, lon2), ...]
        station_names (list of str): Names corresponding to station_coords

    Returns:
        pd.DataFrame: obs_df with new column 'Station'
    """
    obs_df = obs_df.copy()
    obs_lats = obs_df['LAT'].values
    obs_lons = obs_df['LON'].values
    
    station_array = np.array(station_coords)  # shape (n_stations, 2)
    
    assigned_stations = []
    for lat, lon in zip(obs_lats, obs_lons):
        dists = (station_array[:,0] - lat)**2 + (station_array[:,1] - lon)**2
        nearest_idx = np.argmin(dists)
        assigned_stations.append(station_names[nearest_idx])
    
    obs_df['Station'] = assigned_stations
    return obs_df

station_coords = list(zip(lats, lons))
obs = infer_station_names(obs, station_coords, station_names)

In [7]:
# ==============================
# CHECK DUPLICATE OBSERVATIONS
# ==============================

def delete_duplicate_observations(obs_df):
    """
    Check for duplicate observations in the DataFrame.

    Parameters:
        obs_df (pd.DataFrame): Observation DataFrame

    Returns:
        pd.DataFrame: DataFrame with duplicate observations replaced by mean values
    """
    def mean_observation(group):
        if len(group) > 1:
            print(f"Found {len(group)} duplicate observations for Station {group.name[0]} at {group.name[1]}")
        row = group.iloc[0].copy()
        row['AVE_ACTIV'] = group['AVE_ACTIV'].mean()
        row['AVE_ACTIV_ERR'] = group['AVE_ACTIV_ERR'].mean()
        return row

    obs_df_no_duplicates = obs_df.groupby(['Station', 'COLLECT_STOP'], as_index=False, sort=False).apply(mean_observation).reset_index(drop=True)

    return obs_df_no_duplicates

obs = delete_duplicate_observations(obs)

/tmp/ipykernel_706527/3358697507.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  obs_df_no_duplicates = obs_df.groupby(['Station', 'COLLECT_STOP'], as_index=False, sort=False).apply(mean_observation).reset_index(drop=True)


In [8]:
# ==============================
# CREATE input_subexp.dat
# ==============================

input_df = obs.copy()

input_df['Metric'] = obs['NAME']
    
# Use COLLECT_STOP as Date (end of measurement)
input_df['Date'] = pd.to_datetime(obs['COLLECT_STOP']).dt.strftime('%Y-%m-%d %H:%M')
    
# Map values and uncertainties
input_df['Value'] = obs['AVE_ACTIV']
input_df['Uncertainty'] = obs['AVE_ACTIV_ERR']
input_df['MDC Value'] = 1.E-11      # Placeholder MDC value
    
# Prepare output list
rows = []
obs_rows = []
    
for station in station_names:
    # Select all rows corresponding to this station
    tmp_obs_rows = input_df[input_df['Station'] == station].copy()
    tmp_obs_rows['Entity'] = station  # Set Entity to station name
    tmp_obs_rows.sort_values(by='Date', inplace=True)
    obs_rows.append(tmp_obs_rows)

    # Select and order columns for output
    station_rows = input_df[input_df['Station'] == station].copy()
    station_rows['Entity'] = station  # Set Entity to station name
    station_rows = station_rows[['Entity', 'Metric', 'Date', 'Value', 'Uncertainty', 'MDC Value']]
    station_rows.sort_values(by='Date', inplace=True)
    rows.append(station_rows)
    
# Concatenate all stations in the order given
df_out = pd.concat(rows, ignore_index=True) # To input_subexp.dat
obs_df = pd.concat(obs_rows, ignore_index=True) # For future use
    
df_out.to_csv(OUTDIR + "/input_subexp.dat", index=False, float_format="%.8E")

In [9]:
# ==============================
# LOAD SRM DATA
# ==============================

srm_data = xr.open_dataset("SRM.nc")

In [ ]:
# ==============================
# REGRID SRM DATA TO REGULAR GRID
# ==============================

def regrid_to_regular_grid(ds, steplon=0.25, steplat=0.25):
    """
    Regrid 2D spatial data from irregular (y_i, x_i) grid to regular lon-lat grid.
    Instead of interpolation, bins data by summing all points that fall into each grid cell.
    """
    # Get 2D coordinate arrays
    lons_2d = ds['lons_map'].values  # shape (y_i, x_i)
    lats_2d = ds['lats_map'].values  # shape (y_i, x_i)
    
    lon_min, lon_max = np.nanmin(lons_2d), np.nanmax(lons_2d)
    lat_min, lat_max = np.nanmin(lats_2d), np.nanmax(lats_2d)
    
    # Align grid to multiples of step size
    lon_min_aligned = np.floor(lon_min / steplon) * steplon
    lon_max_aligned = np.ceil(lon_max / steplon) * steplon
    lat_min_aligned = np.floor(lat_min / steplat) * steplat
    lat_max_aligned = np.ceil(lat_max / steplat) * steplat
    
    # Create regular 1D grids (bin edges)
    new_lons = np.arange(lon_min_aligned, lon_max_aligned + steplon, steplon)
    new_lats = np.arange(lat_min_aligned, lat_max_aligned + steplat, steplat)
    
    print(f"Lon range: {lon_min:.2f} to {lon_max:.2f} → {lon_min_aligned:.2f} to {lon_max_aligned:.2f}")
    print(f"Lat range: {lat_min:.2f} to {lat_max:.2f} → {lat_min_aligned:.2f} to {lat_max_aligned:.2f}")
    print(f"Grid bins: {len(new_lons)} lons x {len(new_lats)} lats")
    
    # Flatten coordinates
    lons_flat = lons_2d.flatten()
    lats_flat = lats_2d.flatten()
    
    # Determine which bin each point falls into
    lon_bins = np.digitize(lons_flat, new_lons) - 1
    lat_bins = np.digitize(lats_flat, new_lats) - 1
    
    # Regrid srm_conc: for each (sensor_i, ts, td), bin the 2D map
    srm_data = ds['srm_conc'].values  # shape (sensor_i, ts, td, y_i, x_i)
    n_sensor, n_ts, n_td = srm_data.shape[:3]
    
    new_srm = np.zeros((n_sensor, n_ts, n_td, len(new_lats)-1, len(new_lons)-1))
    
    for s in range(n_sensor):
        for t in range(n_ts):
            for d in range(n_td):
                print(f"Binning sensor={s}, ts={t}, td={d}")
                # Extract 2D map for this time and sensor
                data_2d = srm_data[s, t, d, :, :]  # shape (y_i, x_i)
                values = data_2d.flatten()
                
                # Bin data by summing values in each grid cell
                for i in range(len(values)):
                    lon_idx = lon_bins[i]
                    lat_idx = lat_bins[i]
                    
                    # Check bounds
                    if 0 <= lon_idx < len(new_lons)-1 and 0 <= lat_idx < len(new_lats)-1:
                        new_srm[s, t, d, lat_idx, lon_idx] += values[i]
    
    # Create coordinate grids (cell centers)
    lon_centers = (new_lons[:-1] + new_lons[1:]) / 2
    lat_centers = (new_lats[:-1] + new_lats[1:]) / 2
    new_lon_grid, new_lat_grid = np.meshgrid(lon_centers, lat_centers)
    
    return new_srm, new_lon_grid, new_lat_grid

srm_data_regridded, lon_grid, lat_grid = regrid_to_regular_grid(srm_data)
print(f"\nRegridded shape: {srm_data_regridded.shape}")
print(f"Grid shape: {lon_grid.shape}")
np.save(os.path.join(OUTDIR, 'SRM_grid.npy'), srm_data_regridded)
np.save(os.path.join(OUTDIR, 'lon_grid.npy'), lon_grid)
np.save(os.path.join(OUTDIR, 'lat_grid.npy'), lat_grid)

In [14]:
# ==============================
# CREATE GRIDDED SRS FILES FOR EACH OBSERVATION
# ==============================

def create_gridded_file_for_input_row(idx, input_data, lon_grid, lat_grid, srm_data_regridded, output_dir=OUTDIR):
    """
    Create gridded srs file for given input data row.
    Returns a list of rows with lon, lat, ts_index, srm_conc sorted by ts_index, lon, lat
    """
    station = input_data['Entity']
    metric = input_data['Metric']
    end_date = pd.to_datetime(input_data['Date'])
    value = input_data['Value']
    uncertainty = input_data['Uncertainty']
    mdc_value = input_data['MDC Value']

    station_index = station_names.index(station)
    srm_slice = srm_data_regridded[station_index]
    td_indices = np.where(pd.to_datetime(srm['td'].values) == end_date)
    td_index = -1 if len(td_indices) == 0 or len(td_indices[0]) == 0 else td_indices[0][0]
    if td_index == -1:
        srm_slice = np.array([[[]]]) # empty slice
        start_date = end_date
    else:
        srm_slice = srm_slice[:, td_index, :, :]  # shape (ts, lat, lon)
        ts_times = pd.to_datetime(srm['ts'].values)

        start_date = end_date - pd.Timedelta(days=1)
        nonzero_ts_idx = np.where(np.any(srm_slice != 0, axis=(1, 2)))[0]
        if len(nonzero_ts_idx) > 0:
            start_date = ts_times[nonzero_ts_idx[0]]

        time_mask = (ts_times <= end_date) & (ts_times >= start_date)
        srm_slice = srm_slice[time_mask, :, :]  # shape (ts in range, lat, lon)

    # Extract data as list of rows
    rows = []
    for ts_index in range(srm_slice.shape[0]):
        for lat_idx in range(srm_slice.shape[1]):
            for lon_idx in range(srm_slice.shape[2]):
                row = {
                    'lon': lon_grid[lat_idx][lon_idx],
                    'lat': lat_grid[lat_idx][lon_idx],
                    'ts_index': ts_index,
                    'srm_conc': float(srm_slice[ts_index][lat_idx][lon_idx])
                }
                rows.append(row)

    header = {'lat': lats[station_index], 'lon': lons[station_index], 
              'start_date': start_date.strftime('%Y%m%d'), 'start_time': start_date.strftime('%H'), 
              'end_date': end_date.strftime('%Y%m%d'), 'end_time': end_date.strftime('%H'),
              'scale_factor': int(SCALING_FACTOR), 'nsimhours': srm_slice.shape[0] * 24,
              'outputfreq': 24, 'aveTime': 24, 'dx': 0.25, 'dy': 0.25, 'station': station}
    
    os.makedirs(os.path.join(output_dir,'data'), exist_ok=True)
    filename = f'srs_{station}_{idx}_{end_date.strftime("%Y%m%d%H")}.txt.gz'
    filepath = os.path.join(output_dir,'data', filename)
    with gzip.open(filepath, 'wt') as f:
        print(f'{header["lat"]} {header["lon"]} {header["start_date"]} {header["start_time"]} {header["end_date"]} {header["end_time"]} {header["scale_factor"]} {header["nsimhours"]} {header["outputfreq"]} {header["aveTime"]} {header["dx"]} {header["dy"]} "{header["station"]}"', file=f)
        for row in rows:
            print(f'{row["lat"]:.4f} {row["lon"]:.4f} {row["ts_index"]} {row["srm_conc"]:.8E}', file=f)
    print(filename, file = open(os.path.join(output_dir, 'srsfilelist_subexp.dat'), 'a'))

In [ ]:
for idx in range(len(obs_df)):
    print(idx)
    create_gridded_file_for_input_row(idx, obs_df.iloc[idx], lon_grid, lat_grid, srm_data_regridded, output_dir=OUTDIR)

In [16]:
# ==============================
# DELETE EMPTY FILES AND UPDATE LISTS
# ==============================

data_dir = os.path.join(OUTDIR, 'data')

def is_empty_gz_file(file_path):
    """
    Check if a .txt.gz file contains no srs data.
    """
    try:
        with gzip.open(file_path, 'rt') as f:
            lines = f.readlines()
            return len(lines) <= 1  # Empty or header-only
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return False

def delete_empty_files_and_update_lists():
    """
    1. Find all .txt.gz files that contain only header
    2. Delete empty files
    3. Remove corresponding lines from srsfilelist_subexp.dat
    4. Remove corresponding rows from input_subexp.dat
    """
    
    # Find empty files
    empty_files = []
    for filename in sorted(os.listdir(data_dir)):
        if filename.endswith('.txt.gz'):
            filepath = os.path.join(data_dir, filename)
            if is_empty_gz_file(filepath):
                empty_files.append(filename)
                print(f"Found empty file: {filename}")
    
    print(f"\nTotal empty files found: {len(empty_files)}\n")
    
    # Delete empty files
    for filename in empty_files:
        filepath = os.path.join(data_dir, filename)
        os.remove(filepath)
        print(f"Deleted: {filename}")
    
    # Update srsfilelist_subexp.dat
    srsfilelist_path = os.path.join(OUTDIR, 'srsfilelist_subexp.dat')
    if os.path.exists(srsfilelist_path):
        with open(srsfilelist_path, 'r') as f:
            lines = f.readlines()
        
        updated_lines = [line for line in lines if line.strip() not in empty_files]
        
        with open(srsfilelist_path, 'w') as f:
            f.writelines(updated_lines)
        
        print(f"\nUpdated {srsfilelist_path}: removed {len(lines) - len(updated_lines)} entries")
    
    # Update input_subexp.dat (remove rows corresponding to empty files)
    input_dat_path = os.path.join(OUTDIR, 'input_subexp.dat')
    if os.path.exists(input_dat_path):
        import pandas as pd
        df = pd.read_csv(input_dat_path)
        
        # Extract indices from empty filenames to identify rows
        # Format: srs_STA_lat_lon_INDEX_datetime.txt.gz
        empty_indices = []
        for filename in empty_files:
            # Parse filename to extract index
            parts = filename.replace('.txt.gz', '').split('_')
            idx = int(parts[-2])  # Index is second to last after removing extension
            empty_indices.append(idx)
        
        # Remove rows where index matches empty file indices
        original_len = len(df)
        df = df[~df.index.isin(empty_indices)]
        
        df.to_csv(input_dat_path, index=False, float_format="%.8E")
        print(f"Updated {input_dat_path}: removed {original_len - len(df)} rows")

delete_empty_files_and_update_lists()

Found empty file: srs_STA_19.292_166.611_61_2011033000.txt.gz
Found empty file: srs_STA_19.292_166.611_62_2011033100.txt.gz
Found empty file: srs_STA_21.522_-157.995_101_2011033000.txt.gz
Found empty file: srs_STA_21.522_-157.995_102_2011033100.txt.gz
Found empty file: srs_STA_28.220_-177.370_40_2011033000.txt.gz
Found empty file: srs_STA_28.220_-177.370_41_2011033100.txt.gz
Found empty file: srs_STA_38.673_-121.362_122_2011033000.txt.gz
Found empty file: srs_STA_38.673_-121.362_123_2011033100.txt.gz
Found empty file: srs_STA_48.651_-123.445_143_2011033000.txt.gz
Found empty file: srs_STA_48.651_-123.445_144_2011033100.txt.gz
Found empty file: srs_STA_53.050_158.780_19_2011033000.txt.gz
Found empty file: srs_STA_53.050_158.780_20_2011033100.txt.gz
Found empty file: srs_STA_55.337_-160.491_80_2011033000.txt.gz
Found empty file: srs_STA_55.337_-160.491_81_2011033100.txt.gz

Total empty files found: 14

Deleted: srs_STA_19.292_166.611_61_2011033000.txt.gz
Deleted: srs_STA_19.292_166.611_6